### Import libraries

In [27]:
import flopy
import os
import flopy.utils.binaryfile as bf
import numpy as np
from osgeo import gdal
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from tempfile import TemporaryDirectory
from pathlib import Path

### Create MODFLOW model simulation file

In [2]:
name = "peat_testing_0"
temp_dir = TemporaryDirectory()
workspace = temp_dir.name

sim = flopy.mf6.MFSimulation(sim_name = name, 
                            sim_ws=workspace)


### Set time discretization 

Set simulation for:
- 10 stress periods
- 365 (days?) of stress period length
- 1 time steps in a stress period
- 1 multiplier length of stress period

In [3]:
tdis = flopy.mf6.ModflowTdis(
    sim, nper=10, perioddata=[[365,1,1] for _ in range(10)]
)

package_name = peat_testing_0.tdis
filename = peat_testing_0.tdis
package_type = tdis
model_or_simulation_package = simulation
simulation_name = peat_testing_0

Block dimensions
--------------------
nper
{internal}
(10)


Block perioddata
--------------------
perioddata
{internal}
(rec.array([(365., 1, 1.), (365., 1, 1.), (365., 1, 1.), (365., 1, 1.),
           (365., 1, 1.), (365., 1, 1.), (365., 1, 1.), (365., 1, 1.),
           (365., 1, 1.), (365., 1, 1.)],
          dtype=[('perlen', '<f8'), ('nstp', '<i4'), ('tsmult', '<f8')]))



### Set iterative model solver (ims) and gwf package (?)

In [6]:
ims = flopy.mf6.ModflowIms(sim)
gwf = flopy.mf6.ModflowGwf(sim, modelname=name, save_flows=True)

gwf

name = peat_testing_0
model_type = gwf6
version = mf6
model_relative_path = .


### Read raster input files for grid discretization

In [30]:
# Raster paths
demPath = "../testing_peat_module/Raster//DEMNAS_1012-53_v1.0_48s_fill_100m_clip.tif"

# Open files
demDs = gdal.Open(demPath)

# Get data as arrays
demData = demDs.GetRasterBand(1).ReadAsArray()

### Set grid

In [21]:
# spatial discretization object
ztop = demData
zbot = 0
nlay = 2
nrow = demDs.RasterYSize
ncol = demDs.RasterXSize


dis = flopy.mf6.ModflowGwfdis(gwf, nlay, nrow, ncol, ztop, top=zbot, botm=zbot)


117

### Create initial condition package

In [22]:
# Initial head
start = 3

ic = flopy.mf6.modflow.mfgwfic.ModflowGwfic(gwf, pname= "ic", strt=start)

### Set hydraulic conductivity (property flow)

In [25]:
# Set hydraulic conductivity value
hk = [1E-6]
# Create node property flow package
npf = flopy.mf6.modflow.mfgwfnpf.ModflowGwfnpf(
    gwf, pname="npf", icelltype=1, k=hk, save_flows=True
)

npf

package_name = npf
filename = peat_testing_0.npf
package_type = npf
model_or_simulation_package = model
model_name = peat_testing_0

Block options
--------------------
save_flows
{internal}
(True)


Block griddata
--------------------
icelltype
{constant 1}

k
{constant 1e-06}



### Add RIV package

In [33]:
#Add the drain package (DRN) to the MODFLOW model
river_path = "../testing_peat_module/Raster/Stream_DEMNAS.tif"
river = gdal.Open(river_path)
list = []
for i in range(river.shape[0]):
    for q in range(river.shape[1]):
        if river[i,q] == 1:
            list.append([0,i,q,ztop[i,q],0.001]) #layer,row,column,elevation(float),conductance
rivDrn = {0:list}

drn = flopy.modflow.ModflowDrn(gwf, stress_period_data=rivDrn)

river

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000027F45662DF0> >